In [1]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from prophet import Prophet

In [38]:
df = pd.read_csv('../monica/monica_weather_2011-2016.csv')
df['date'] = pd.to_datetime(df['de-date'], format='%d.%m.%Y')

In [39]:
df.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid,date
0,01.01.2011,-6.82,-8.20,-5.45,3.80,2.20,2.78,97.94,2011-01-01
1,02.01.2011,-4.67,-5.85,-3.02,5.48,2.43,0.81,95.94,2011-01-02
2,03.01.2011,-8.42,-13.24,-5.63,2.41,3.33,0.23,96.19,2011-01-03
3,04.01.2011,-11.02,-13.81,-9.68,4.04,2.99,3.20,97.75,2011-01-04
4,05.01.2011,-13.10,-14.60,-10.60,3.73,4.06,0.75,98.50,2011-01-05


In [40]:
def train_prophet(col):
    df_ = df[['date', f'{col}']].copy()
    df_ = df_.rename(columns={"date": "ds", f'{col}': "y"})

    model = Prophet(interval_width=0.95)
    model.fit(df_)
    future = model.make_future_dataframe(periods=365, include_history=False)
    forecast = model.predict(future)
    
    return forecast.yhat_lower.to_numpy(), forecast.yhat.to_numpy(), forecast.yhat_upper.to_numpy()

In [41]:
cols = ['tavg', 'tmin', 'tmax', 'wind', 'globrad', 'precip','relhumid']

In [42]:
test = pd.read_csv('../monica/reanalysis_2017.csv')

In [43]:
test.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
0,01.01.2017,-3.85,-6.03,-1.72,3.96,2.14,0.90,98.50
1,02.01.2017,-1.89,-3.75,-0.85,5.43,1.92,0.57,96.75
2,03.01.2017,-3.72,-9.02,-1.72,5.07,2.04,0.90,95.38
3,04.01.2017,-6.60,-9.16,-5.06,5.27,3.57,3.88,97.56
4,05.01.2017,-4.36,-6.16,-2.20,3.25,3.15,1.17,97.38


In [44]:
for col in cols:
    test[f'{col}_lower'], test[f'{col}_'], test[f'{col}_upper'] = train_prophet(col)

08:24:07 - cmdstanpy - INFO - Chain [1] start processing
08:24:07 - cmdstanpy - INFO - Chain [1] done processing
08:24:09 - cmdstanpy - INFO - Chain [1] start processing
08:24:09 - cmdstanpy - INFO - Chain [1] done processing
08:24:11 - cmdstanpy - INFO - Chain [1] start processing
08:24:11 - cmdstanpy - INFO - Chain [1] done processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] done processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:13 - cmdstanpy - INFO - Chain [1] done processing
08:24:13 - cmdstanpy - INFO - Chain [1] start processing
08:24:13 - cmdstanpy - INFO - Chain [1] done processing
08:24:14 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmdstanpy - INFO - Chain [1] done processing


In [45]:
# Lower
lower = test[[
    'de-date', 'tavg_lower', 'tmin_lower', 'tmax_lower', 'wind_lower', 'globrad_lower', 'precip_lower', 'relhumid_lower'
]]
lower = lower.rename(columns={
    "tavg_lower":"tavg", "tmin_lower":"tmin", "tmax_lower":"tmax", "wind_lower":"wind", "globrad_lower":"globrad",
    "precip_lower":"precip", "relhumid_lower":"relhumid"
})
lower.to_csv(
    '../monica/prophet_2017_lower.csv', index=False
)

# Avg
avg = test[[
    'de-date', 'tavg_', 'tmin_', 'tmax_', 'wind_', 'globrad_', 'precip_', 'relhumid_'
]]
avg = avg.rename(columns={
    "tavg_":"tavg", "tmin_":"tmin", "tmax_":"tmax", "wind_":"wind", "globrad_":"globrad",
    "precip_":"precip", "relhumid_":"relhumid"
})
avg.to_csv(
    '../monica/prophet_2017.csv', index=False
)

# Upper
upper = test[[
    'de-date', 'tavg_upper', 'tmin_upper', 'tmax_upper', 'wind_upper', 'globrad_upper', 'precip_upper', 'relhumid_upper'
]]
upper = upper.rename(columns={
    "tavg_upper":"tavg", "tmin_upper":"tmin", "tmax_upper":"tmax", "wind_upper":"wind", "globrad_upper":"globrad",
    "precip_upper":"precip", "relhumid_upper":"relhumid"
})
upper.to_csv(
    '../monica/prophet_2017_upper.csv', index=False
)